<a href="https://colab.research.google.com/github/rzl-ds/gu511_exam/blob/master/2020.final.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>

# GU511 FINAL EXAM (2020)

## goal

in this midterm exam we will cover all the material covered in lectures so far in 2020, including:

+ new material
    + web scraping and `REST` `api`s
    + `aws` `iam`
    + databases
    + `aws` `rds`
    + `aws` `redshift`
    + `gpu` analytics
    + deep learning including `tensorflow` and `keras`
    + `hadoop` and `spark`
+ first-half material
    + `aws` `ec2`
    + `ssh`
    + `linux`
    + `git`
    + `python`
    + `conda`
    + `docker`

## a note on "intermediate products"

the questions and answers in this exam build on eachother:

+ exercise 7 asks you to write `sql` that will upload a dataset to a redshift cluster, and exercise 8 asks you write queries on that uploaded data
+ exercise 2 asks you to write `python` code that will generate a list of downloadable files, and exercise 5 asks you to do something with that list of files

I don't want the inability to do one exercise to block you from doing the next, so in every instance where that is possible I have created a publically available intermediate product that you can use instead, and provided you with the necessary code to do it. **you should not be blocked on any one exercise due to inability to complete another**. if you feel that is not so, reach out to me asap!

## a note on the s3 submission bucket

in exercise 1 you will access a sepcial `s3` bucket we will call your "final submission bucket". every file submitted for this final **MUST** be submitted into **THAT** bucket -- do not use any of the `s3` buckets we have created previously in this class.

in exercise1 we will specifically talk about a flag `--bucket-owner-full-control` that you must provide every time you upload your files. please remember to use that flag for every file you upload to your submission bucket; we cannot access the files otherwise.

## method of delivery

you will submit most of the results of your exam via `s3`. we have created a bucket for you (see exercise 1 below) and you will be pushing files there. **do not** push files to your homework submission buckets we've used in previous weeks!

summary:

| exercise | deliverable | method of delivery | points |
|----------|-------------|--------------------|--------|
| 1 | a file `test.txt` | uploaded to your `s3` **final** (not hw) submission bucket | 5 |
| 2 | a file `urls.py` | uploaded to your `s3` final submission bucket | 15 |
| 3 | a file `exercise1_ec2_details.png` | uploaded to your `s3` final submission bucket | 5 |
| 4 | a file `exercise2_ec2_w_s3_role.png` | uploaded to your `s3` final submission bucket | 5 |
| 5 | a file `usaspending2s3.py` | uploaded to your `s3` final submission bucket | 15 |
| 6 | a file `copy_usaspending_to_postgres.psql` | uploaded to your `s3` final submission bucket | 10 |
| 7 | a file `s3toredshift.sql` | uploaded to your `s3` final submission bucket | 15 |
| 8 | a file `usaspending_queries.sql` | uploaded to your `s3` final submission bucket | 15 |
| 9 | a file `mapreduce_termination.sh` | uploaded to your `s3` final submission bucket | 10 |
| 10 | a `github` `pull request` from a `fork`ed `repo` | will be observe on `github` | 15 |
| -1 | cleanup | nothing to submit | 5 |

total points: 110

<div style="border: 1px solid lightgrey;">

# exercise 0: honor code

while you are welcome to reference course notes, stack overflow, online documentation, and all the other resources of the world, there is one resource which you *CANNOT* use: your fellow classmates.

you are expected to complete this exam **completely independently**. as this exam will be unsupervised, you are (as always) expected to abide by the guidelines laid out by [the honor council](https://honorcouncil.georgetown.edu/), and summarized in the student pledge:

> In pursuit of the high ideals and rigorous standards of academic life I commit myself to respect and to uphold the Georgetown University honor system:
> 
> To be honest in every academic endeavor, and
> 
> To conduct myself honorably, as a responsible member of the Georgetown community as we live and work together.

<div style="border: 1px solid lightgrey;">

# exercise 1: access your `s3` final submission bucket

I have created an `s3` bucket for each of you to submit your answers to the final. it is named

`2020-fall-gu511-[YOUR_GU_ID]`


## 1.1: verify you can read

verify that you can list the contents of that directory with

```sh
# this must be run with your aws account credentials!
aws s3 ls 2020-fall-gu511-[YOUR_GU_ID]
```

you should see one file named `dbcredentials.txt`:

```
2020-12-07 12:07:48         12 dbcredentials.txt
```

verify that you can download and read that file with

```sh
aws s3 cp s3://2020-fall-gu511-[YOUR_GU_ID]/dbcredentials.txt .
cat dbcredentials.txt && echo # print a new line so it's not "hidden"
```


## 1.2: verify you can write

next we will test that you can upload files to that bucket. note: we have to do something special here: we have to set an `acl` (permissions, not ligaments) value. an `a`ccess `c`ontrol `l`ist is something we've set silently on everything we've created so far. in order to make sure that *I* (the bucket owner) can read what *you* (the `s3` file object owner) have created, you have to explicitly grant me that permission. do so with the extra flag `--acl bucket-owner-full-control`:

```sh
echo "hello world" > test.txt
aws s3 cp test.txt s3://2020-fall-gu511-[YOUR_GU_ID]/ --acl bucket-owner-full-control
```

ultimately, you can verify that this worked by going to `https://s3.console.aws.amazon.com/s3/object/2020-fall-gu511-[YOUR_GU_ID]?region=us-east-1&prefix=test.txt` (note: you must fill in `[YOUR_GU_ID]` with, you know...). you should see something like:

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1cgjB7MIOC-XYhPlMdGmC3SC3ZM8EdzK9"></div>

##### we will verify that a `test.txt` file was written to your `s3` final submission bucket

<div style="border: 1px solid lightgrey;">

# exercise 2: access a list of downloadable `zip` files

head over to [the `usaspending.gov` award data archive page](https://www.usaspending.gov/download_center/award_data_archive) we visited in a previous homework assignment.

the page contains a form which allows us to specify a federal agency (default is "All"), an award type (default is "Contracts") and a year (default is 2020). the basic layout is like this (obviously years and dates will be more recent):

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1YrjWErf_J-bE5Fz_25mdxG0cvnHCjuI2"></div>

previously we downloaded one of those files -- `FY2020_All_Contracts_Full_20201108.zip`. today our goal is to download every `FYYYYY_All_Contracts_Full_YYYYMMDD.zip` file. to do that, we *could* click through the form for every year and download every `zip` `url` by hand... but we have better tools!


## 2.1: figuring out the `api` endpoint we want to use

it would be great if we could find a programatic way to list the monthly files we want to download. if we could programmatically access the `url`s of those `zip` archives, we could do everything much faster...

you'll be shocked to find out that *we can*!

first, collect some info from the webpage itself. once you've loaded that award data archive page, open your browser's developer tools and navigate to the tab which shows all `request`s and `response`s made by your browser. if your browser is

+ google chrome, go to the "Network" tab and click the "preserve logs" checkbox
+ firefox, go to the "Network" tab and click the "persist logs" checkbox
+ IE, go to the "Network" tab and toggle the "Clear entries on navigate" icon (details [here](https://docs.microsoft.com/en-us/previous-versions/windows/internet-explorer/ie-developer/samples/dn255004%28v=vs.85%29))

click on the "Fiscal Year" drop-down and change it to 2020, then click the "Apply" button and see what happens in the Network tab of your browser's developer tools window.

+ what `url`s are `request`ed?
+ what type of `request` (`GET`, `POST`, etc) are those?
+ what is the `response`?
+ do any of the `response`s have the information we are looking for?

one of the `request`s we made lines up with one of the endpoints of [the `usaspending` `api`](https://api.usaspending.gov/docs/endpoints) -- specifically, one of the [download endpoints](https://github.com/fedspendingtransparency/usaspending-api/tree/master/usaspending_api/api_contracts/contracts/v2/bulk_download). if called correctly, it will respond with a `json` blob that contains all of the download information we need.

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1NsCNY3lU1FQMy5GTMxU5rmNrA-WkiImW"></div>


## 2.2: a function to get the `json` data we need

using the `python` `requests` library and the information you can glean from the developer tools and the `api` documentation, fill in the following function which takes a year and returns a `url` for downloading a `FYYYYY_All_Contracts_Full_YYYYMMDD.zip` archive of award data. save this to a file named `urls.py`.

```python
import requests

def get_zip_url(year):
    """given the year of interest, find the current url of the zip file
    which contains contract data for *all* agencies for that year
    
    args:
        year (int): the year of contract information we want
        
    returns:
        str: the url of the zip archive for that year's contract data
        
    """
    resp = #-----------------#
           # FILL THIS IN!!! #
           #-----------------#
            
    j = resp.json()
    
    # somewhere within that response's json payload is the url -- get it!
    # make sure you get the *FULL* file and not the *DIFF* file
    url = #-----------------#
          # FILL THIS IN!!! #
          #-----------------#
    
    return url


def zip_url_range(year_0, year_1):
    """a simple helper function for iterating over a range of years.
    this will be inclusive of both endpoints.
    
    args:
        year_0 (int): the first year to get
        year_1 (int): the last year to get
    
    """
    return [get_zip_url(year) for year in range(year_0, year_1 + 1)]
```

if all goes according to plan, you should be able to run

```python
get_zip_url(2021)
```

and the `url` it spits out is the one given by the first link in the table on [the award archive page](https://www.usaspending.gov/#/download_center/award_data_archive). at the time of writing, that is `https://files.usaspending.gov/award_data_archive/20201202.zip`

similarly, you should be able to run

```python
zip_url_range(2019, 2021)
```

and get a list containing three urls


##### upload your updated `urls.py` file to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 3: create an `ec2` instance


## 3.1: image details

we are going to use the list of urls we just generated in exercise 2 to do some bulk download from usaspending, unzipping, and uploading to `s3` (exercise 5). before we do that, though, let's create a new `ec2` instance to do everything on

you have a choice to make for the instance type:

+ you can use the **free** instance type `t2.micro`, but it will be **slow** (there is a task that will take about 2 hours and 45 minutes to run in the background (you don't have to wait on it to go forward))
+ you can use the **for charge** (0.384 USD per hour) instance type `m5.2xlarge`, but it will be **much faster** (that same task will take about 30 minutes)

it should have the following properties:

+ `ami`: "Amazon Linux 2 AMI (HVM), SSD Volume Type" and 64 bit x86
+ type: `t2.micro` (free and slow) or `m5.2xlarge` (for charge and fast)
+ hard drive disk size: 20GiB (the default of 8 won't cut it)


## 3.2: setting a `python` environment

after the launch has been completed, `ssh` in to your machine and create a `miniconda` environment with the following:

```sh
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
sh Miniconda3-latest-Linux-x86_64.sh
# answer the questions!
rm Miniconda3-latest-Linux-x86_64.sh
exit
```

now, re-connect to your instance and verify that your `python` version is correct:

```sh
python --version
```

you should have version 3.8.5 (or higher)

now, also install `boto3` with

```sh
conda install -y boto3
```


## 3.3: pics or it didn't happen

from the web console, take a screenshot of your new `ec2` instance's details menu. for example, mine looks like

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1hZtYhBRWskKLAMiIBTAWcX5Iq2-3RKxB"></div>

be sure your screenshot includes at least the items highlighted in the red box

save that snapshot as `exercise1_ec2_details.png` (you can use a different image format (e.g. `gif`, `jpeg`) if that is easier; just make sure the base name is the same).

##### upload that `exercise1_ec2_details.png` to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 4: create an `ec2`-service `iam` `role`

using the `aws` web console, create a new `role` with the following properties

1. it should be an `aws` service role which will be used by the `ec2` service
1. it will have the `AmazonS3FullAccess` permission policy attached
1. it will be named `gu511-final-ec2-s3-role`

back in the `ec2` service, find your final `ec2` server you created in exercise 2 and attach this role to it (Actions > Security > Modify IAM Role)

switch from the Details tab to the Security tab and take *another* screenshot of this window, making sure to include the `IAM role` field

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1O3kW_5_aAt59LNJm_H1OV8I8xkWorpus"></div>

this time save that picture with the name `exercise2_ec2_w_s3_role.png`

##### upload that `exercise2_ec2_w_s3_role.png` to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 5: download, unzip, and upload files to `s3`

let's create a single `python` function which can download the `zip` files for `usaspending`, unzip them, and upload them to `s3`. the downloading and unzipping code will be provided for you (but explained, in 5.2 and 5.3 below). you will be responsible for filling in the upload of the created files using the `boto3` library (5.4).


## 5.1: getting a list of `zip` files to download

the code we defined in the previous exercise will be used to identify the list of `zip` files we should download. if it is working as expected, we should be able to run (in the directory that contains `urls.py`)

```python
import urls

zip_urls = urls.zip_url_range(2001, 2021)
```

if you didn't complete exercise 4 yet, you can still get a file `urls.py` which will work for now/ please do the following: in a `bash` shell, run

```sh
wget https://s3.amazonaws.com/shared.rzl.gu511.com/urls.py
```

and verify it works by opening a `python` session and running

```python
import urls

zip_urls = urls.zip_url_range(2001, 2021)

assert len(zip_urls) == 20
```


## 5.2: downloading a `zip` file

downloading files is possible with the `requests` library but a little messy and frankly slower than I'm willing to live with for big files -- why not use a faster `bash` option like `wget`? we will use the `python` `subprocess` module to call a `bash` command from *within* our `python` code.

for example, take the last `zip_url`

```python
zip_urls[-1]
```

at the time of writing, this was https://files.usaspending.gov/award_data_archive/FY2020_All_Contracts_Full_20201108.zip

we can easily download that from a `bash` command prompt on our `aws` `linux` `ec2` instances with

```sh
wget --quiet https://files.usaspending.gov/award_data_archive/FY2020_All_Contracts_Full_20201108.zip
```

to use the `subprocess` module, we simply need to build our command strings up as if we were calling them from the `bash` prompt and then split them up based on the spaces within them (done for us in "the right way" by the `shlex` module). the above `bash` command could be done from within `python` via

```python
import shlex
import subprocess

cmd = 'wget --quiet https://files.usaspending.gov/award_data_archive/FY2020_All_Contracts_Full_20201108.zip'
cmd = shlex.split(cmd)
subprocess.run(cmd)
```


## 5.3: unzipping a downloaded `zip` file

each `zip` archive we have downloaded is (internally) a collection of 1M-record-`csv`s representing the contract actions for the downloaded year. the `zipfile` library in `python` is sufficient for extracting these `zip` files into the component `csv` files.

if we had downloaded the `FY2020_All_Contracts_Full_20201108.zip` archive file as in 5.2, we could now run

```python
import zipfile

with zipfile.ZipFile('FY2020_All_Contracts_Full_20201108.zip') as zfp:
    zfp.extractall()
```

this would create files in the same directory as `FY2020_All_Contracts_Full_20201108.zip` that are the contents of that `zip` file.

we can find the names of those `csv` files we have unzipped with the `glob` library:

```python
import glob

unzipped_csvs = glob.glob('./*All_Contracts_Full*.csv')
```


## 5.4: uploading unzipped `csv` files to `s3`

finally, something for you to do!


### 5.4.1: creating an `s3` bucket for uploading these files

in the `aws` web console, go to the `s3` service page and create a new bucket named `[YOUR GU ID]-gu511-final-usaspending`. leave all defaults


### 5.4.2: filling in `boto3` code

given the name of one of the unzipped `csv` files obtained via 5.2 and 5.3, fill in the following `python` code to upload that one file to your `s3` bucket you just created

```python
import boto3

BUCKET_NAME = '[YOUR GU ID]-gu511-final-usaspending'

def csv_to_s3(fcsv, region_name='us-east-1', bucket_name=BUCKET_NAME):
    """given the file name of a csv file, upload it to my s3 bucket.
    we will put each file in a sub-directory of our bucket named
    "raw", so that the full Key value will be
    
        Key = f'raw/{fcsv}'
    
    note: we don't need to deal with permissions or group names here
    because we are using an iam role on our `ec2` server which has
    full s3 permissions. we *do* still have to set the region, though!
    stick with the default (us-east-1)
    
    args:
        fcsv (str): path to the csv file we want to upload
        region_name (str): the aws region for the boto3 session we are 
            about to create
        bucket_name (str): the name of the s3 bucket into which we are
            loading our csvs
    
    returns:
        none
    
    """
    Key = f'raw/{fcsv}'
    #-----------------#
    # FILL THIS IN!!! #
    #-----------------#
```


## 5.5: bringing it all together

update the `csv_to_s3` function in the code below and save the results as `usaspending2s3.py`

```python
import datetime
import glob
import os
import shlex
import subprocess
import zipfile

import boto3
import requests


BUCKET_NAME = '[YOUR GU ID]-gu511-final-usaspending'


def csv_to_s3(fcsv, region_name='us-east-1', bucket_name=BUCKET_NAME):
    """given the file name of a csv file, upload it to my s3 bucket.
    we will put each file in a sub-directory of our bucket named
    "raw", so that the full Key value will be
    
        Key = f'raw/{fcsv}'
    
    note: we don't need to deal with permissions or group names here
    because we are using an iam role on our `ec2` server which has
    full s3 permissions. we *do* still have to set the region, though!
    stick with the default (us-east-1)
    
    args:
        fcsv (str): path to the csv file we want to upload
        region_name (str): the aws region for the boto3 session we are 
            about to create
        bucket_name (str): the name of the s3 bucket into which we are
            loading our csvs
    
    returns:
        none
    
    """
    Key = f'raw/{fcsv}'
    #-----------------#
    # FILL THIS IN!!! #
    #-----------------#


def main():
    t0 = datetime.datetime.now()
    
    # get the list of urls to download
    zip_urls = requests.get(
        'https://s3.amazonaws.com/shared.rzl.gu511.com/zip_list_key.json'
    ).json()

    for (i, zip_url) in enumerate(zip_urls):
        # downloading...
        cmd = f'wget --quiet {zip_url}'
        cmd = shlex.split(cmd)
        print(f'downloading {zip_url}...')
        subprocess.run(cmd)
        print('download complete')

        # unzipping...
        zipfile_name = os.path.basename(zip_url)
        print(f'unzipping {zipfile_name}...')
        with zipfile.ZipFile(zipfile_name) as zfp:
            zfp.extractall()
        print('unzipped')
        
        # removing the zip file
        os.remove(zipfile_name)
        
        # new csv files
        unzipped_csvs = glob.glob('*All_Contracts_Full*.csv')
        print(f'{len(unzipped_csvs)} new csv files')
        for fcsv in unzipped_csvs:
            print(f'uploading file {fcsv} to s3...')
            csv_to_s3(fcsv)
            print('uploaded')
        
            # removing the csv file
            os.remove(fcsv)
        
        t1 = datetime.datetime.now()
        print(f'we have completed {i + 1} out of {len(zip_urls)} years')
        print(f'in {t1 - t0}')
    
    t1 = datetime.datetime.now()
    print(f'task complete in {t1 - t0}')


if __name__ == '__main__':
    main()
```

if everything works as planned, you should be able to push all the `usaspending` data to your `s3` bucket with this script. 

this entire process takes about **2 hours and 45 minutes** on a `t2.micro` and about **30 minutes** on an `m4.2xlarge`, so you may not want to stay and watch it run. you need to "background" the command with `nohup`:

```sh
nohup python -u usaspending2s3.py &
```

when you execute the code that way, you can close your terminal and come back without killing the running process. you can check in on the state of that process (now running in the background) with

```sh
tail -f nohup.out
```

at any time.

***NOTE: if you created the more expensive `ec2` instance to do this assignment, please shut it off!!!***


##### upload your file `usaspending2s3.py` to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 6: `\copy`ing those `csv`s to `rds` 

we just uploaded almost 100GiB of files to `s3`, and it would be nice to put them some place. one options is in a traditional relational database like `postgres`.


## 6.1: logging in

I have created a user name and table for each of you on my personal `postgres` database. you can connect to it with the `psql` command. first and foremost, on your `ec2` instance you created in exercise 2, install the `psql` command

```sh
sudo yum install -y postgresql
```

after installation is complete, use the `psql` command to connect to my `postgres` `rds` with the command

```sh
GUID=[FILL THIS IN!!]
psql -h gu511-fall-2020-final-psql.cdmknaubrmaw.us-east-1.rds.amazonaws.com -U $GUID --dbname postgres
```

you will be prompted for your password -- that is in the file `dbcredentials.txt` which I uploaded to your `s3` finals bucket. you can view it with the series of commands

```sh
aws s3 cp s3://2020-fall-gu511-$GUID/dbcredentials.txt .
cat dbcredentials.txt && echo
# optionally, remove it
rm dbcredentials.txt
```

you can verify that you've logged in and that the table exists by executing

```sql
select count(*) from [YOUR GU ID]_usaspending;
```

you should see 0 rows


## 6.2: getting data to insert

rather than insert any one (let alone all!) of those large files, let's just get one much smaller file and insert that as a test of our `\copy` `psql` skills. download the insertable records `csv` file on your `ec2` instance with

```sh
wget https://s3.amazonaws.com/shared.rzl.gu511.com/usaspending_dev.csv -O /tmp/usaspending_dev.csv
```


## 6.3: inserting data

use the `\copy` command to copy the local (to your finals `ec2` instance) `/tmp/usaspending_dev.csv` file into the table named `[YOUR GU ID]_usaspending`. write the command you use to do this to a file `copy_usaspending_to_postgres.psql`

a successful command will yield 100 records in the `[YOUR GU ID]_usaspending` table:

```
gu511db=> select count(*) from [YOUR GU ID]_usaspending ;
 count 
-------
   100
(1 row)
```


***note: you are done with your finals `ec2` instance now, so feel free to terminate it if you have completed the assignments up to this one***

##### upload `copy_usaspending_to_postgres.psql` to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 7: `copy`ing those `csv`s to `redshift`

our entire collection of `csv`s saved out on `s3` is now about 81.2 GiB -- well within the reach of a typical `rdbms` like `postgres` or `redshift`. because our data is in `s3` and we can copy directly from `s3` to `redshift` (and we cannot say the same for `postgres`) let's go with `redshift` for our main data warehouse for this `usaspending` data.

we would like to `copy` the files in our `s3` bucket (named `[YOUR GU ID]-gu511-final-usaspending`) to `redshift`.


## 7.1: re-connect to or re-build your `redshift` cluster from a previous homework

in the `012_dbs_5_redshift` lecture, we walked through the creation of a `redshift` cluster and installation of `datagrip`. additionally, on a previous homework you were asked to execute a few queries in that `redshift` cluster using `datagrip`.

if that cluster (the one you created) is still up and running, use it again for this excercise. if not, please create a new `redshift` cluster following the steps laid out in that `012_dbs_5_redshift.ipynb` lecture, and connect to it using `datagrip`.

**note**: please re-use the `allow_redshift_s3_read_role` `iam` role for this cluster (created in lecture [here](https://colab.research.google.com/github/rzl-ds/gu511/blob/master/012_dbs_5_redshift.ipynb#scrollTo=YokUQWSImXOv)). we will again be copying files from `s3` so this will be essential


## 7.2: alternatives to your `s3` bucket

we want to push about 115 GiB of `csv` files from `s3` to `redshift`. if you were unable to complete exercise 4 and have not populated a bucket named `[YOUR GU ID]-gu511-final-usaspending` with the `usaspending` data, you may instead use my publicly available bucket `rzl5-gu511-final-usaspending` which has files located in `s3://rzl5-gu511-final-usaspending/raw`.


## 7.3: the `create table` command

I initially considered asking you to determine the schema for this 277-column table, but the fact of the matter is that it's a long, stupid exercise and would be mostly a waste of your time. so I have saved the `create table` command for you as a file at https://s3.amazonaws.com/shared.rzl.gu511.com/redshift_create_usaspending.sql. download that file and execute it while connected to your `redshift` cluster.


## 7.4: copy command

let's load `csv` information from `s3` into a `redshift` table. this is very similar to how we created the `trainpositions` table in a previous homework, with one twist: we are loading all of the items in an `s3` bucket instead of just one file. you may need to [refer to the `redshift` documentation](https://docs.aws.amazon.com/redshift/latest/dg/copy-parameters-data-source-s3.html)

a properly running command can take **1 hour and 45 minutes** to run. if you haven't seen an error through a minute, it's running. go do anything else.

when you have a `copy` query expression that succesfully copies records from `s3` to `redshift`, save that expression to a file `s3toredshift.sql`


##### upload `s3toredshift.sql` to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 8: some simple `sql` queries


## 8.1: a `redshift` for all of us

hopefully you were able to complete exercise 7 and have access to a `redshift` cluster with a `usaspending` table in it. if not, though, you can connect to my `redshift` cluster.

+ jdbc: `jdbc:redshift://rzl5-gu511-final-redshift.csneoatl4kff.us-east-1.redshift.amazonaws.com:5439/dev`
+ the `username` and `password` were included in the announcement on canvas

if you run into any issues connecting, please reach out.


## 8.2: querying the `usaspending` data

let's do just a few simple things to make sure our `redshift` cluster is functional. fill in the `sql` queries describe below and save the resulting file as `usaspending_queries.sql`.

for reference, I have included the results I received for the `csv`s which had an "as of" date `2020-11-08` (this could change during the run of the exam, and I will try to update the text below to include those values as well)

```sql
/*  understaning awards ---------------------------------------------------- */

/*  a single award (aka contract) is defined by a pair of values:

        award_id_piid, parent_award_id_piid

    every award has an award_id_piid, but there are a handful of records that
    don't have parent_award_id_piid values (they are the highest-level contracts
    and other contracts roll up under them)

    the query below will show a handful of these pairs of identifying id values
    among our many records. note that many have parent_award_id_piid values that
    are null!
*/
SELECT DISTINCT award_id_piid,
                parent_award_id_piid
FROM usaspending
LIMIT 100
;

/*  we can calculate whether or not a contract has a parent by checking the
    value of parent_award_id_piid

        parent_award_id_piid = ''

    indicates there is no parent, and

        parent_award_id_piid != ''

    indicates there is a parent

    we can easily determine how many conracts have parents
*/
SELECT has_parent,
       COUNT(*) AS ct
FROM (SELECT DISTINCT award_id_piid,
                      parent_award_id_piid,
                      CASE
                          WHEN parent_award_id_piid = '' THEN 0
                          ELSE 1
                      END AS has_parent
      FROM usaspending)
GROUP BY has_parent
;
/*  for "as of" 2020-11-08, this is

1 38202037
0 14282946
*/


/*  modifications of awards ------------------------------------------------- */

/*  for any single award (aka contract) defined by

        award_id_piid, parent_award_id_piid

    a modification is any distinct value of the field

        modification_number

    find the 5 awards which had the highest number of modifications
*/
SELECT
    -- FILL THIS IN!!!
FROM
    -- FILL THIS IN!!!
GROUP BY
    -- FILL THIS IN!!!
ORDER BY
    -- FILL THIS IN!!!
LIMIT 5;
/*  for "as of" 2020-11-08, this is

N0002409C2107  ""  3992
GS35F0511T     ""  3898
USZA2203C0056  ""  3507
F4261098C0001  ""  3178
N0002403C2101  ""  2672

*/


/*  find the 5 awards *which had parents* which had the highest number of
    modifications
*/
SELECT
    -- FILL THIS IN!!!
FROM
    -- FILL THIS IN!!!
WHERE
    -- FILL THIS IN!!!
GROUP BY
    -- FILL THIS IN!!!
ORDER BY
    -- FILL THIS IN!!!
LIMIT 5;
/*  for "as of" 2020-11-08, this is

0004        N0002404D4409  679
SD21        F3365701D2050  531
0017        W912ER04D0004  488
FB00        N6927200D3170  431
NNJ07JF12D  NAS598144      383

*/

/*  award monetary values --------------------------------------------------- */

/*  the monetary value of a contract is given by the field

        current_total_value_of_award

    find the highest monetary value of any single award
*/
SELECT
    -- FILL THIS IN!!!
FROM
-- FILL THIS IN!!!
;
/*  for "as of" 2020-11-08, this is

    1000202900000

    (that's 1 trillion dollars, 1,000,202,900,000)
*/

/*  find the lowest *non-zero* monetary value of any single award  */
SELECT
    -- FILL THIS IN!!!
FROM
    -- FILL THIS IN!!!
WHERE
    -- FILL THIS IN!!!
;
/*  for "as of" 2020-11-08, this is

    0.01
*/

/*  contract termination ---------------------------------------------------- */

/*  when a modification is made to a contract the modifying agent is required to
    provide a code describing why that action was taken. that code is saved as
    field

        action_type

    there ar a handful of action types, but among them are a few "bad"
    action_type values:

        TERMINATE FOR DEFAULT (COMPLETE OR PARTIAL)
        TERMINATE FOR CAUSE

    for every award, determine whether or not that award ever included either or
    the above `action_type` values. do this by using two sql functions:

        + use a `case` statement to create a binary variable for each record
          with a value of 1 if the `action_type` is one of the two above and 0
          otherwise
        + use the `max` function to determine whether or not there were any 1
          indicators for a given award

    the result of this query will be a list of awards (award_id_piid,
    parent_award_id_piid pairs) and a flag indicating whether there was or was
    not a termination on that award. we will save the result of that query as a
    new table

    NOTE: if you are running on the shared cluster, you probably can't create
    this table because I already did. create your own with a name of
    `[YOUR_GU_ID]_award_terminated`
*/
CREATE TABLE award_terminated AS (
    SELECT award_id_piid,
           parent_award_id_piid,
           (
               -- FILL THIS IN!!!
               ) AS had_termination_action_type
    FROM usaspending
    GROUP BY award_id_piid,
             parent_award_id_piid
);

/*  if this worked, you can run the following  */
SELECT had_termination_action_type,
       COUNT(*)
FROM award_terminated
GROUP BY had_termination_action_type
;
/*  for "as of" 2020-11-08, this is

0  52470836
1  14147
*/

```

##### upload `usaspending_queries.sql` to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 9: using `emr` to process `s3` spending data

at the end of the previous exercise we developed a `sql` query to determine whether or not any one of the contract actions defined for a single contract (uniquely identified by a pair of fields, `award_id_piid` and `parent_award_id_piid`) were of a "bad" type. let's replicate that using `python` scripts and `hadoop` streaming in an `emr` cluster!


## 9.1: starting a cluster

create a new `emr` cluster using the "quick options" menu. you can clone the one we made in class, or you can create a new one -- if you go for a new cluster, the only things to change from the default configurations are:

+ pick a descriptive name
+ under the "Security and access" section, pick an "EC2 key pair" that you have access to.
+ make sure that the security group for the master node has a rule allowing you to make `ssh` connections from your personal IP address.


## 9.2: getting the code

log in to the `emr` master node and download the `usaspending_mapreduce` code. make sure you're getting the `branch` named **`final`**, not the `master` branch, with the following code:

```sh
sudo yum install -y git
git clone -b final https://github.com/rzl-ds/usaspending_mapreduce.git
cd usaspending_mapreduce
```

and then put the `mapper.py` and `reducer.py` files into `hdfs` with

```sh
hadoop fs -put mapper.py
hadoop fs -put reducer.py
```


## 9.3: executing a map reduce job

following the example in the lecture and the last homework, execute a `hadoop-streaming` job using the provided `mapper.py` and `reducer.py` files.


### 9.3.1: development

for a development stage, use a small input

+ `input`: your smallest `csv` in `s3`
    + as of writing, this would be `s3://[YOUR GU ID]-gu511-final-usaspending/raw/FY2004_All_Contracts_Full_20201108_3.csv`
+ `output`: `s3://rzl5-gu511-final-usaspending/terminations-test/`
    + this is the directory your output will be written to
+ you will also need to specify the `files`, `mapper`, and `reducer` arguments to `hadoop jar`

if this works as expected, you will see the standard `map` `reduce` log messages in your terminal, and in the end you will have a directory `s3://rzl5-gu511-final-usaspending/terminations-test/` that contains `part-****` files with the output of the map-reduce job.


### 9.3.2: doing the full job

the ultimate goal is to run this process for your *entire* `s3` bucket `s3://[YOUR GU ID]-gu511-final-usaspending/raw`, and to write the result to a directory `s3://[YOUR GU ID]-gu511-final-usaspending/terminations`. 

this whole process will take about ***1 hour and 45 minutes***

please write the `hadoop jar` command you use to run a `hadoop-streaming` job with `mapper.py` and `reducer.py` and save it as `mapreduce_termination.sh`


##### upload `mapreduce_termination.sh` to your final `s3` submission bucket. don't forget the `--acl bucket-owner-full-control`

<div style="border: 1px solid lightgrey;">

# exercise 10: `fork`ing and `pull request`ing

one final `github` exercise until freedom!

in two of the last homework exercises we covered special `github`-specific features for collaborating on a `repository` via special `github` features -- either by direct commits to the `master` `branch` referencing issues (automatic closure of issues) or by creating a new `branch` in a repository and then initiating a `pull request` and a `github`-based merge commit.

there is yet another way of contributing to repositories and collaborating on `github`, and that is `pull request`s from `fork`ed repositories.


## 10.1: `fork`ing

a `fork` or a repository is a special `github` concept. basically, you are a creating your own personal copy of someone else's `github` repository in a way that `github` knows how to track (it "remembers" the original repository you copied).

I have created a repository https://github.com/rzl-ds/gu511_students where there is one file for every one of your `github` account names. the content of that file right now is simply

```txt
I'm almost done!
```

in the top right corner of the `github` page is the `fork` button

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1NChn8mklEgdeJWvV3slBl8w-Yb9Wb2Me"></div>

when you click on that button you will create your own copy of this repository under *your* `github` account.

`fork` it!


## 10.2: making a change in your `fork`ed `repo`

you're no longer "almost" done, you *are* done. let's update that file to reflect that! please edit the file associated with your `github` account name (and **only** that file!). furthermore, let's do it *entirely* on `github` (no `cli` at all).

in your `fork`ed `repo`, click on the link for your `.txt` file. at the top of the window showing the file's contents is a pencil icon which will allow you to edit directly on `github`. click it!

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1tosAKKNttLjT1mofVoka7AOnTg1ciTQ3"></div>

in that editor, update the message to say anything you'd like (as long as it's different).

at the bottom, you are given a form to fill out that mirrors the elements of the command line `commit` we would make.

1. add whatever `commit` message you'd like
1. **click the "Create a new branch"** radio button
1. click "propose file change"
1. **DO NOT** create a pull request from this page
    + confusingly, `github` here is suggesting a `pull request` from your `fork`ed `branch` to your `fork`ed `master` -- that's *not* what we want to do
1. head back to your repository's top level web page

at this point, if everything went according to plan, you'll see something like

<br><div align="center"><img src="http://drive.google.com/uc?export=view&id=1XJr2msCzooJa9iRU49zHqqkRYvGNCPf1"></div>

1. click on "compare & pull request"
1. from the left dropdown boxes, select `rzl-ds/gu511_students` and `master` branch
1. from the right dropdown boxes, select your repo and your recently created `patch` branch
1. click "create pull request"

at this point I will receive a notification that you have asked me (owner of `rzl-ds/gu511_students`) if I will accept your `pull request`. I will be able to reivew it and possibly `merge` it into my original `repo` all from within the comfort of my `chrome` window

##### there is nothing to submit; work will be verified on `github`

<div style="border: 1px solid lightgrey;">

# exercise -1: cleaning up `aws` services 

we've reached the end! now's a good time to go through the following checklist of services we have used in this course and see if you have anything running there you don't want to keep -- they won't be free forever!

+ `ec2`: this includes instances and also elastic ip addresses (you will start paying for them unless you release them)
+ `s3`: please keep your homework and final submission buckets as is for the time being. beyond that, feel free to delete anything you no longer need
+ `rds`: you can kill any database instances you created for this class
    + check for backups / snapshots, too -- delete them if they exist
+ `redshift`: if you have finished the exam, you can shut down any `redshift` clusters you have created
    + check for backups / snapshots, too -- delete them if they exist
+ `emr`: you should definitely shut down any `emr` clusters you have running

<div style="border: 1px solid lightgrey;">